# QRT ENS Challenge Data 2023 - Benchmark

Version 1 - Boosting, Feature engeneering & XGBoost 

## Librairies

In [ ]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer

from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from scipy.stats import spearmanr
import numpy as np
import itertools

import warnings
warnings.filterwarnings('ignore')

## Chargement des données

- `X_train` et `X_test` ont  $35$ colonnes qui représentent les même variables explicatives mais sur des périodes de temps différentes. 

- `X_train` et `Y_train` partagent la même colonne `ID` - chaque ligne a un ID unique associé à un jour et à un pays. 

- La variable cible `TARGET` de `Y_train` correspond à la variation de prix journalière des futures sur l'électricité (maturité 24h).

- **On notera que certaines colonnes ont des valeurs manquantes**.


In [ ]:
# After downloading the X_train/X_test/Y_train .csv files in your working directory:

X_train = pd.read_csv('../data/raw/X_train.csv')
Y_train = pd.read_csv('../data/raw/y_train.csv')
X_test = pd.read_csv('../data/raw/X_test_final.csv')

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

In [ ]:
X_train["TARGET"] = Y_train["TARGET"]

## Feature engineering
The main goal here is to reconstruct some of the lost time dimension to create stationary features.

In general we had added statistics, technical indicators, seasonality, clusters and bag of features.

In [ ]:
def slope(y):
    return np.polyfit(range(len(y)), y, 1)[0] if len(y) > 0 else np.nan

In [ ]:
def add_rolling_statistics(df, variables, windows, countries=['DE_', 'FR_']):
    # Define a slope calculation function
    def slope(y):
        return np.polyfit(range(len(y)), y, 1)[0] if len(y) > 0 else np.nan

    # Calculate rolling statistics for each variable and window
    for var in variables:
        for window in windows:
            for country in countries:
                df[f'{country}{var}_MEAN_{window}D'] = df[f'{country}{var}'].rolling(window=window).mean()
                df[f'{country}{var}_STD_{window}D'] = df[f'{country}{var}'].rolling(window=window).std()
                df[f'{country}{var}_MEDIAN_{window}D'] = df[f'{country}{var}'].rolling(window=window).median()
                df[f'{country}{var}_MIN_{window}D'] = df[f'{country}{var}'].rolling(window=window).min()
                df[f'{country}{var}_MAX_{window}D'] = df[f'{country}{var}'].rolling(window=window).max()
                # Apply the slope function to the rolling window
                df[f'{country}{var}_SLOPE_{window}D'] = df[f'{country}{var}'].rolling(window=window).apply(slope, raw=True)
    return df

In [ ]:
def add_seasonality_features(df):
    days_in_year = 365.25
    df['SIN_YEAR'] = np.sin(2 * np.pi * df['DAY_ID'] / days_in_year)
    df['COS_YEAR'] = np.cos(2 * np.pi * df['DAY_ID'] / days_in_year)
    
    days_in_week = 7
    df['SIN_WEEK'] = np.sin(2 * np.pi * df['DAY_ID'] / days_in_week)
    df['COS_WEEK'] = np.cos(2 * np.pi * df['DAY_ID'] / days_in_week)

    df['SEASON'] = pd.cut(df['DAY_ID'] % 365, bins=[0, 79, 172, 264, 365], labels=[0, 1, 2, 3], right=False).astype(int)
    return df

In [ ]:
def add_energy_source_ratios_and_effects(df):
    for country in ['DE_', 'FR_']:
        for energy_source in ['GAS', 'COAL', 'HYDRO', 'NUCLEAR', 'SOLAR', 'WINDPOW']:
            total_energy = df[f'{country}GAS'] + df[f'{country}COAL'] + df[f'{country}HYDRO'] + \
                           df[f'{country}NUCLEAR'] + df[f'{country}SOLAR'] + df[f'{country}WINDPOW']
            df[f'{country}{energy_source}_RATIO'] = df[f'{country}{energy_source}'] / total_energy

        df[f'{country}WIND_SOLAR'] = df[f'{country}WINDPOW'] + df[f'{country}SOLAR']
        df[f'{country}TEMP_EFFECT'] = df[f'{country}TEMP'] * df[f'{country}CONSUMPTION']
        df[f'{country}WIND_EFFECT'] = df[f'{country}WIND'] * df[f'{country}WINDPOW']
        df[f'{country}SOLAR_EFFECT'] = (df[f'{country}SOLAR'] / df[f'{country}TEMP']).replace([np.inf, -np.inf], np.nan).fillna(0)
    
    return df

In [ ]:
def add_market_features(df):
    for commodity in ['GAS_RET', 'COAL_RET', 'CARBON_RET']:
        df[f'{commodity}_VOLATILITY_7D'] = df[commodity].rolling(window=7).std()
        df[f'{commodity}_VOLATILITY_30D'] = df[commodity].rolling(window=30).std()
        df[f'{commodity}_EMA_30D'] = df[commodity].ewm(span=30, adjust=False).mean()
    
    return df

In [ ]:
def add_custom_features(df):
    # Temporal window and variables for rolling statistics
    windows = [7, 30]
    variables = ['CONSUMPTION', 'GAS', 'COAL', 'HYDRO', 'NUCLEAR', 'SOLAR', 'WINDPOW', 'TEMP', 'RAIN', 'WIND']

    # Calculate rolling statistics and other features
    df = add_rolling_statistics(df, variables, windows)

    # Seasonality Features
    df = add_seasonality_features(df)

    # Energy Source Ratios and Effects
    df = add_energy_source_ratios_and_effects(df)

    # Market Volatility and Moving Averages
    df = add_market_features(df)

    # Ensure all missing data are filled if any new were created
    df.fillna(method='bfill', inplace=True)

    return df

In [ ]:
def feature_engineering(X):
    # Copy the original dataframe to avoid modifying it directly
    df = X.copy()
    
    # Define the country codes
    country_codes = {'DE': 0, 'FR': 1}
    
    # Map country codes to numerical values
    df['COUNTRY'] = df['COUNTRY'].map(country_codes)

    imputer = SimpleImputer(strategy='median')
    df[df.columns] = imputer.fit_transform(df)

    df = add_custom_features(df)
    
    return df

#### *Apply the feature engineering to the df*

In [ ]:
df = feature_engineering(X_train)

In [ ]:
def select_features_based_on_correlation(dataframe, target_column, multicollinear_threshold=0.7, correlation_threshold=0.05):
    # Calculate the Spearman correlation matrix
    corr_matrix = dataframe.corr(method='spearman')
    
    # Identify features that are highly correlated with each other
    # (excluding the target variable correlation)
    high_corr_var = np.where(corr_matrix > multicollinear_threshold)
    high_corr_var = [(corr_matrix.index[x], corr_matrix.columns[y]) 
                     for x, y in zip(*high_corr_var) 
                     if x != y and x < y]
    
    # Extract the names of columns to drop based on multicollinearity
    multicollinear_features = set([item for sublist in high_corr_var for item in sublist])
    
    # Identify features that have a low correlation with the target variable
    low_corr_with_target = corr_matrix[target_column][abs(corr_matrix[target_column]) < correlation_threshold].index.tolist()
    
    # Combine features to drop due to multicollinearity and low correlation with target
    features_to_drop = multicollinear_features.union(low_corr_with_target)
    
    # Determine the final list of features to keep
    features_to_keep = [feature for feature in dataframe.columns if feature not in features_to_drop and feature != target_column]
    
    return features_to_keep

In [ ]:
def select_top_features(features, max_features):
    if len(features) > max_features:
        return features[:max_features]
    else:
        return features

selected_features = []

refined_features = select_features_based_on_correlation(df, 'TARGET', 0.7, 0.05)
top_features = select_top_features(refined_features, 30)
df_reduced = df[top_features + ['TARGET']]
print(f"Selected features for df: {len(df_reduced.columns)}")

## Model and training

#### Use of GridSearch to find optimal hyperparameters

In [ ]:
def train_and_evaluate_catboost(X, y):
    # Define the parameter grid to search over
    param_grid = {
        'depth': [4, 6],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [200],
        'l2_leaf_reg': [3, 7]
    }

    # Determine all combinations
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Set up K-Fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    best_score = -np.inf
    best_params = None

    # Iterate over combinations
    for params in combinations:
        scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]

            model = CatBoostRegressor(silent=True, **params)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            
            score = spearmanr(y_test, predictions).correlation
            scores.append(score)

        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = params

        print(f"Params: {params}, Mean Spearman Correlation: {mean_score}")

    print(f"Best Params: {best_params}, Best Mean Spearman Correlation: {best_score}")
    return best_params

In [ ]:
print(f"Training CatBoost model...")
X = df_reduced.drop('TARGET', axis=1)
y = df_reduced['TARGET'].rank()
best_params = train_and_evaluate_catboost(X, y)

In [ ]:
# Retrain with best params on whole dataset
final_model = CatBoostRegressor(silent=True, **best_params)
final_model.fit(X, y)

## Prepare submission

In [ ]:
X_test_final = pd.read_csv('../data/raw/X_test_final.csv')
df_test = feature_engineering(X_test_final)

In [ ]:
df_test_reduced = df_test[top_features]

In [ ]:
predictions = final_model.predict(df_test_reduced)

In [ ]:
Y_test_submission = X_test_final[['ID']].copy()
Y_test_submission["TARGET"] = predictions

In [ ]:
Y_test_submission.to_csv('submission_catboost.csv', index=False)